In [7]:
from Benchmark import *
from constant import *

In [10]:
def test_in_laptop():
    ROOT = "~/Desktop/intern_project/samza-azure-storage/milestone2/db_data"
    DB_BENCH = "~/Desktop/intern_project/rocksdb/db_bench"
    KEY_SIZE = 20 * B
    VALUE_SIZE = 400 * B
    DATA_SIZE = 100 * MB
    NUM_KEYS = int(DATA_SIZE / (KEY_SIZE + VALUE_SIZE))
    laptopBenchmark = Benchmark(root_dir = ROOT, db_bench = DB_BENCH, key_size = KEY_SIZE, 
                               value_size = VALUE_SIZE, data_size = DATA_SIZE, other_params = "")
    laptopBenchmark.benchmark_all()

In [11]:
test_in_laptop()
#parser = LogParser(res)
#parser.parse_all()
#basic_res_dict = parser.get_basic_res()
#delay_percentiles_dict = parser.get_percentiles()
#print(res)
#print(basic_res_dict)
#print(delay_percentiles_dict)

Setup a new test root. Test root dir ~/Desktop/intern_project/samza-azure-storage/milestone2/db_data/test-976746278
command : ~/Desktop/intern_project/rocksdb/db_bench --benchmarks="fillseq"            --db=~/Desktop/intern_project/samza-azure-storage/milestone2/db_data/test-976746278           --histogram=1           --num=249660           --use_existing_db=0           --key_size=20           --value_size=400            --statistics  
Dump result to file : ~/Desktop/intern_project/samza-azure-storage/milestone2/db_data/test-976746278/performance-testing-log/fillseq-performance-result


FileNotFoundError: [Errno 2] No such file or directory: '~/Desktop/intern_project/samza-azure-storage/milestone2/db_data/test-976746278/performance-testing-log/fillseq-performance-result'